In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
def load_data(filepath):
    return pd.read_csv(filepath)

In [3]:
def split_features_and_target(df):
    X = df.drop(columns=['Strain', 'Rating', 'Description'])
    y = df['Rating']
    return X, y

In [4]:
def preprocess_data(X):
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    return preprocessor

In [5]:
def create_pipeline(preprocessor, model):
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    return pipeline

In [6]:
def train_and_evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2

In [7]:
def main():
    filepath = 'cannabis.csv'
    df = load_data(filepath)
    X, y = split_features_and_target(df)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    preprocessor = preprocess_data(X)
    linear_model = LinearRegression()
    pipeline = create_pipeline(preprocessor, linear_model)
    
    mse, r2 = train_and_evaluate_model(pipeline, X_train, X_test, y_train, y_test)
    print(f"Linear Regression Model - MSE: {mse}, R2: {r2}")

In [8]:
if __name__ == "__main__":
    main()

Linear Regression Model - MSE: 0.5953997507779596, R2: 0.2492950359498981


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

def load_data(filepath):
    return pd.read_csv(filepath)

def split_features_and_target(df):
    X = df.drop(columns=['Strain', 'Rating', 'Description'])
    y = df['Rating']
    return X, y

def preprocess_data(X):
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['object']).columns

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    return preprocessor

def create_pipeline(preprocessor, model):
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    return pipeline

def hyperparameter_tuning(pipeline, X_train, y_train):
    param_grid = {
        'model__alpha': [0.1, 1.0, 10.0, 100.0]
    }

    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

def train_and_evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2

def main():
    filepath = 'cannabis.csv'
    df = load_data(filepath)
    X, y = split_features_and_target(df)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    preprocessor = preprocess_data(X)
    ridge_model = Ridge()
    pipeline = create_pipeline(preprocessor, ridge_model)
    
    best_model = hyperparameter_tuning(pipeline, X_train, y_train)
    
    mse, r2 = train_and_evaluate_model(best_model, X_train, X_test, y_train, y_test)
    print(f"Optimized Ridge Regression Model - MSE: {mse}, R2: {r2}")

if __name__ == "__main__":
    main()

Optimized Ridge Regression Model - MSE: 0.4351558301529688, R2: 0.45133728825996433
